In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('lv2-2307.csv')

In [ ]:
df.info()

#### Q1

In [ ]:
df4 = df.copy()

In [ ]:
from datetime import datetime
df4['DAY'] = df4['SIG_DATE'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%w') )
df4['WEEKEND'] = df4['DAY'].apply(lambda x : 'Yes' if (x=='6')|(x=='0') else 'No')

In [ ]:
q1 = df4['SIGNAL'].quantile(0.25)
q3 = df4['SIGNAL'].quantile(0.75)
iqr = q3 - q1

In [ ]:
df4['OUTLIER'] = df4['SIGNAL'].apply(lambda x : 0 if (x >= q1 - 1.5*iqr) & (x <= q3 + 1.5*iqr) else 1)

In [ ]:
A = len(df4[(df4['WEEKEND'] == 'Yes') & (df4['OUTLIER'] == 1)]) / len(df4[df4['WEEKEND'] == 'Yes'])
B = len(df4[(df4['WEEKEND'] == 'No') & (df4['OUTLIER'] == 1)]) / len(df4[df4['WEEKEND'] == 'No'])

In [ ]:
ans1 = A / B

In [ ]:
print('Q1 Result: ', ans1)
print('Q1 Answer: ', round(ans1,3))

#### Q2

In [ ]:
df5 = df.copy()

In [ ]:
df5_1 = df5[df5['SETUP'] != 0]

In [ ]:
tec_mean = df5_1.groupby(['TECHNOLOGY'])['SETUP'].mean().reset_index()
df5_1 = df5.merge(tec_mean, how='left', on='TECHNOLOGY')

In [ ]:
def aa(x):
    if x[7] == 0:
        x[7] = x[9]
    return x

In [ ]:
df5_2 = df5_1.apply(aa, axis=1)
df5_2.rename(columns = {'SETUP_x':'SETUP'}, inplace=True)
D = df5_2['SETUP'].mean() + 1.96 * df5_2['SETUP'].std() / (len(df5_2) ** 0.5)
C = df5_2['SETUP'].mean() - 1.96 * df5_2['SETUP'].std() / (len(df5_2) ** 0.5)
B = df5['SETUP'].mean() + 1.96 * df5['SETUP'].std() / (len(df5) ** 0.5)
A = df5['SETUP'].mean() - 1.96 * df5['SETUP'].std() / (len(df5) ** 0.5)

In [ ]:
ans2 = abs(D-A)

In [ ]:
print('Q2 Result: ', ans2)
print('Q2 Answer: ', round(ans2, 3))

#### Q3

In [ ]:
df6 = df.copy()

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
t_list = df6['TECHNOLOGY'].unique()

In [ ]:
def bb (x):
    if x[3] <= 0:
        x[3] = x[2] * b
    return x

In [ ]:
# sklearn
for t in t_list:
    tmp = df6[df6['TECHNOLOGY'] == t]
    tmp1 = tmp[tmp['SPEED'] > 0]
    lr = LinearRegression(fit_intercept=False)
    lr.fit(tmp1[['SIGNAL']], tmp1[['SPEED']])
    b = lr.coef_[0][0]
    b = round(b, 3)
    tmp = tmp.apply(bb, axis=1)
    tmp.set_index(['SIG_ID'], inplace=True)
    df6.set_index(['SIG_ID'], inplace=True)
    df6.loc[tmp.index, 'SPEED'] = tmp['SPEED']
    df6.reset_index(inplace=True)
    print(t)
    print(b)

In [ ]:
import statsmodels.api as sm
# OLS 로 계산
for t in t_list:
    tmp = df6[df6['TECHNOLOGY'] == t]
    tmp1 = tmp[tmp['SPEED'] > 0]
    ols = sm.OLS(tmp1[['SPEED']], tmp1[['SIGNAL']]).fit()
    b = ols.params[0]
    b = round(b, 3)
    tmp = tmp.apply(bb, axis=1)
    tmp.set_index(['SIG_ID'], inplace=True)
    df6.set_index(['SIG_ID'], inplace=True)
    df6.loc[tmp.index, 'SPEED'] = tmp['SPEED']
    df6.reset_index(inplace=True)
    print(t)
    print(b)

In [ ]:
df6['TARGET'] = df6['MOS'].apply(lambda x : 1 if x >= 4 else 0)

In [ ]:
train = df6[df6['SIG_ID'] % 4 != 0]
test = df6[df6['SIG_ID'] % 4 == 0]
train_x = train.drop(columns = ['SIG_ID', 'SIG_DATE', 'RESULT', 'TECHNOLOGY','SETUP','MOS','TARGET'])
train_y = train[['TARGET']]
test_x = test.drop(columns = ['SIG_ID', 'SIG_DATE', 'RESULT', 'TECHNOLOGY','SETUP','MOS','TARGET'])
test_y = test[['TARGET']]

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(solver = 'lbfgs', C = 100000, random_state = 1234)
lr.fit(train_x, train_y)

In [ ]:
test_y['prd'] = lr.predict(test_x)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
ans3 = accuracy_score(test_y['TARGET'], test_y['prd'])

In [ ]:
print('Q3 Result: ', ans3)
print('Q3 Answer: ', round(ans3,3))